# Artificial Neural Network (ANN) - Basic Example

## Predicting Customer Churn (Banking)

The exercise applies an Artifical Neural Network (ANN) using Tensorflow/Keras to predict customer churn at a bank. 

This notebook illustrates model training using Train-Test-Split and predicts customer churn based on new information.

#### Import Required Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

Using TensorFlow backend.


#### Import Dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### Encode Categorical Data

In [3]:
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/Users/michaelcascio/.pyenv/versions/3.7.2/envs/data-tools/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/michaelcascio/.pyenv/versions/3.7.2/envs/data-tools/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


#### Split Dataset into Training and Test Datasets

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#### Scale Features

In [5]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#### Create the Artificial Neural Network (ANN)

In [6]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


#### Fit the ANN to the Training Dataset

In [7]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 2s 248us/step - loss: 0.4864 - acc: 0.7955
Epoch 2/100
8000/8000 [==============================] - 2s 192us/step - loss: 0.4306 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 220us/step - loss: 0.4263 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 206us/step - loss: 0.4221 - acc: 0.8011
Epoch 5/100
8000/8000 [==============================] - 2s 200us/step - loss: 0.4187 - acc: 0.8231
Epoch 6/100
8000/8000 [==============================] - 2s 200us/step - loss: 0.4166 - acc: 0.8252
Epoch 7/100
8000/8000 [==============================] - 2s 202us/step - loss: 0.4144 - acc: 0.8296 0s - loss: 0.
Epoch 8/100
8000/8000 [==============================] - 2s 200us/step - loss: 0.4132 - acc: 0.8297
Epoch 9/100
8000/8000 [==============================] - 2s 221us/step - loss: 0.4115 - acc: 0.8326
Epoch 10/100
8000/8000 [==============

#### Make Predictions Based on Test Dataset (Probability of churn > 0.5 means customer will churn)

In [10]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

#### Evaluate Model Accuracy on Test Dataset Using Confusion Matrix

In [12]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1519,   76],
       [ 200,  205]])

In [14]:
test_accuracy = ((1519+205)/(1519+205+76+200))*100
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 86.20%


#### Predict a Single New Observation (Will this new customer churn?)

New Customer Information: 
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000

#### New customter information is input as a 2-dimensional np.array, transformed with the same scaler previously fit to the training data. Values are input as floats to avoid warning. First two values being 0,0 denote the one-hot-encoding for 'France'. Fourth value being 1 denotes one-hot-encoding for Male. The prediction that the customer will churn is false, meaning they are not expected to churn (probability < 0.5)

In [21]:
new_prediction = classifier.predict(sc.transform(np.array([[0.,0.,600.,1.,40.,3.,60000.,2.,1.,1.,50000.]])))
new_prediction = (new_prediction>0.5)
new_prediction

array([[False]])